# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure all your path constants are **relative to** ***DATA_DIR*** and **NOT hard-coded** in your code.

In [0]:
!pip install whoosh
!pip install pytrec_eval
!pip install wget

In [0]:
import wget
wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/government.zip", "government.zip")

'government.zip'

In [0]:
!unzip government.zip

Archive:  government.zip
replace government/topics-with-full-descriptions.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace government/gov.topics? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace government/gov.qrels? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace government/documents/61/G00-61-2800209? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace government/documents/61/G00-61-1192048? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace government/documents/61/G00-61-1118212? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace government/documents/61/G00-61-1118212? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: government/documents/61/G00-61-1118212  
  inflating: government/documents/61/G00-61-0749882  
  inflating: government/documents/61/G00-61-2230501  
  inflating: government/documents/61/G00-61-0680698  
  inflating: government/documents/61/G00-61-0551387  
  inflating: government/documents/61/G00-61-2575433  
  inflating: government/documents/6

In [0]:
import numpy as np
import os,os.path
from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
from pathlib import Path
import tempfile
import subprocess
import pytrec_eval
import wget
import shutil
import nltk
from nltk.stem import *
from whoosh import qparser
from whoosh import scoring
# imports
# Put all your imports here

In [0]:
DATA_DIR = "government"
DOCUMENTS_DIR = os.path.join(DATA_DIR, "documents")
TOPIC_FILE = os.path.join(DATA_DIR, "gov.topics")
QRELS_FILE = os.path.join(DATA_DIR, "gov.qrels")
#
# Put other path constants here
#

## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 (a) here [markdown cell]

MAP might be appropriate for measuring search 
system performance for government website.

### Q1 (b): Provide answer to Q1 (b) here [markdown cell]

For web search, most searchers don't look past the first page, so having more relevant documents of the top-ranked results would be helpful for them to find what they want, that's why MAP is the most appropriate measure.


## Question 2

### Q2 (a): Write your code below

In [0]:
def createIndex(schema):
    # Generate a temporary directory for the index
    indexDir = tempfile.mkdtemp()

    # create and return the index
    return index.create_in(indexDir, schema)

In [0]:
# first, define a Schema for the index
mySchema_Q2 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))

# now, create the index at the path INDEX_DIR based on the new schema
INDEX_Q2 = createIndex(mySchema_Q2)

In [0]:
def addFilesToIndex(indexObj, fileList):
    # open writer
    writer = writing.BufferedWriter(indexObj, period=None, limit=1000)

    try:
        # write each file to index
        for docNum, filePath in enumerate(fileList):
            with open(filePath, "r", encoding="utf-8") as f:
                fileContent = f.read()
                writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if ((docNum+1) % 1000 == 0):
                    print("already indexed:", docNum+1)
        print("done indexing.")

    finally:
        # close the index
        writer.close()

In [0]:
# Build a list of files to index
filesToIndex = [str(filePath) for filePath in Path(DOCUMENTS_DIR).glob("**/*") if filePath.is_file()]

In [0]:
addFilesToIndex(INDEX_Q2, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [0]:
# define a query parser for the field "file_content" in the index
QP_Q2 = QueryParser("file_content", schema=INDEX_Q2.schema)
SEARCHER_Q2 = INDEX_Q2.searcher()

In [0]:
def pyTrecEval(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            topicResults = searcher.search(topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, {'map'})

    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    for query_id, query_measures in results.items():
        for measure, value in query_measures.items():
            if measure == "runid":
              continue
            print_line(measure, query_id, value)
    for measure in query_measures.keys():
        if measure == "runid":
              continue
        print_line(
            measure,
            'all',
            pytrec_eval.compute_aggregated_measure(
                measure,
                [query_measures[measure]
                 for query_measures in results.values()]))

### Q2 (b): Provide answer to Q2 (b) here [markdown cell]

In [0]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q2, SEARCHER_Q2) 

map                      1       0.0000
map                      2       0.0000
map                      4       0.0312
map                      6       0.0000
map                      7       0.0000
map                      9       0.0000
map                      10      0.1667
map                      14      0.2500
map                      16      0.0000
map                      18      1.0000
map                      22      0.2000
map                      24      1.0000
map                      26      0.1111
map                      28      0.0000
map                      all     0.1971


The baseline Whoosh system has a MAP score of 0.1971 of all query topics.

### Q2 (c): Provide answer to Q2(c) here [markdown cell]

From the above table, topics 18 and 24 perform best with score of 1.0, and topics 1,2,6,7,9,16,28 perform worst with score of 0.0 .

## Question 3

In [0]:
def printRelName(topicFile, qrelsFile, queryParser, searcher, id):
  with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()
  for topic in topics:
        topic_id, topic_phrase = tuple(topic.split(" ", 1))
        if topic_id == id:
          print("---------------------------Topic_id and Topic_phrase----------------------------------")
          print(topic_id, topic_phrase)
          topicQuery = queryParser.parse(topic_phrase)
          topicResults = searcher.search(topicQuery, limit=None)
          print("---------------------------Return documents----------------------------------")
          for (docnum, result) in enumerate(topicResults):
              score = topicResults.score(docnum)
              print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
          print("---------------------------Relevant documents----------------------------------")
          with open(qrelsFile, 'r') as f_qrel:
            qrels = f_qrel.readlines()
            for i in qrels:
              qid, _, doc, rel = i.rstrip().split(" ")
              if qid == id and rel == "1":
                print(i.rstrip())

In [0]:
with open(TOPIC_FILE, "r") as f:
    print(f.read())

1 mining gold silver coal
2 juvenile delinquency
4 wireless communications
6 physical therapists
7 cotton industry
9 genealogy searches
10 Physical Fitness
14 Agricultural biotechnology
16 Emergency and disaster preparedness assistance
18 Shipwrecks
19 Cybercrime, internet fraud, and cyber fraud
22 Veteran's Benefits
24 Air Bag Safety
26 Nuclear power plants
28 Early Childhood Education



In [0]:
printRelName(TOPIC_FILE, QRELS_FILE, QP_Q2, SEARCHER_Q2,"4")

---------------------------Topic_id and Topic_phrase----------------------------------
4 wireless communications
---------------------------Return documents----------------------------------
4 Q0 G00-99-2247765 0 16.449155 test
4 Q0 G00-85-1525415 1 13.364613 test
4 Q0 G00-05-1218739 2 12.956314 test
4 Q0 G00-09-0774298 3 11.781349 test
4 Q0 G00-56-4151981 4 11.367248 test
4 Q0 G00-21-2229498 5 10.743958 test
4 Q0 G00-98-4068688 6 10.464865 test
4 Q0 G00-47-2117970 7 10.213356 test
4 Q0 G00-67-0152545 8 8.392871 test
4 Q0 G00-06-1757034 9 6.431556 test
4 Q0 G00-78-2551063 10 3.955775 test
4 Q0 G00-84-0274223 11 2.068438 test
---------------------------Relevant documents----------------------------------
4 0 G00-03-2855342 1
4 0 G00-36-1275993 1
4 0 G00-47-2117970 1
4 0 G00-65-0162935 1


In [0]:
with open(DOCUMENTS_DIR+'/36/G00-36-1275993','r') as f:
    print(f.read())

http://www.ntia.doc.gov/ntiahome/threeg/


                                3G Wireless
     __________________________________________________________________

   INTRODUCTION: "Wireless" Internet: What the 3G Challenge Means for U.S.
   Competitiveness

   10-05-01: New Plan to Identify Spectrum for Advanced Wireless Mobile
   Services (3G)

   Text of 6-Sep-2001 Letter from Commerce Secretary Evans to
   Congressional Leaders, Proposing Legislation to Shift Statutory
   Spectrum Auction Date. (Proposed legislative language)

   Acting Assistant Secretary William T. Hatch testified on July 31, 2001,
   before the Senate Commerce, Science, and Transportation Communications
   Subcommittee on spectrum management and 3rd generation wireless
   service.

   Acting Assistant Secretary William T. Hatch testified before the
   Subcommittee on Telecommunications and the Internet of the House Energy
   and Commerce Committee on 3G wireless issues on July 24, 2001.

   Text of Secretary Evans J

In [0]:
with open(DOCUMENTS_DIR+'/99/G00-99-2247765','r') as f:
    print(f.read())

http://w3.antd.nist.gov/wire_tel.shtml

   [USEMAP:wire_te.gif]


   Goals
   Horizontal
   To help the US cellular communications industry test the wideband CDMA
   components (cdma2000 and W-CDMA systems) of the IMT-2000 family of
   standards for third-generation wireless systems over different traffic
   and wireless channel quality conditions
   To facilitate transmission of video signals over 3G wireless systems.

   Technical Approach
   Horizontal
     * Develop SPW models for the forward link, reverse link, and
       multi-carrier mode of the cdma2000 system
     * Measure the performance of the cdma2000 system over different ITU
       channel models and in face of interference from other users
       communicating multimedia information
     * Develop a partitioning method and a multi-priority transmission
       method to send video over 3G wireless systems

   FY02 Plans
   Horizontal
     * Provide industry customers with as needed consulting to understand
       and use

### Q3 (a): Provide answer to Q3 (a) here [markdown cell]

With query topic 4 "wireless communications", we have the above results. There are 4 relevant documents and we retreived 12 documents, but only one of them(G00-47-2117970) is relevant.

Relevant document "G00-36-1275993" is not retreived, so it is false negative. And retreived document "G00-05-1218739" is not relevant, so it is false positive.   

We can see that relavent file "G00-36-1275993" doesn't return. Because in this file ,topic "wireless communications" shows as "Wireless","Communications" and "Telecommunications". So we need to do LowercaseFilter and Stemfilter.

And to lower the number of false positives, we need to do add StopFilter to filter stop words. 

### Q3 (b): Write your code below

In [0]:
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
from whoosh.analysis import Filter
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [0]:
# download required resources
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
Analyzer_Q3 = RegexTokenizer() | LowercaseFilter() | StopFilter()| CustomFilter(LancasterStemmer().stem)
# first, define a Schema for the index
mySchema_Q3 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = Analyzer_Q3))

# now, create the index at the path INDEX_DIR based on the new schema
INDEX_Q3 = createIndex(mySchema_Q3)


In [0]:
addFilesToIndex(INDEX_Q3, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [0]:
# define a query parser for the field "file_content" in the index
QP_Q3 = QueryParser("file_content", schema=INDEX_Q3.schema)
SEARCHER_Q3 = INDEX_Q3.searcher()

In [0]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q3, SEARCHER_Q3) 

map                      1       0.0000
map                      2       0.5000
map                      4       0.5375
map                      6       0.0000
map                      7       0.0000
map                      9       0.0556
map                      10      0.2500
map                      14      1.0000
map                      16      0.0000
map                      18      1.0000
map                      19      0.5000
map                      22      0.0357
map                      24      1.0000
map                      26      0.0771
map                      28      0.2262
map                      all     0.3455


In [0]:
printRelName(TOPIC_FILE, QRELS_FILE, QP_Q3, SEARCHER_Q3,"4")

---------------------------Topic_id and Topic_phrase----------------------------------
4 wireless communications
---------------------------Return documents----------------------------------
4 Q0 G00-36-1275993 0 15.113268 test
4 Q0 G00-47-2117970 1 14.550367 test
4 Q0 G00-99-2247765 2 14.344555 test
4 Q0 G00-00-1958915 3 13.225127 test
4 Q0 G00-85-1525415 4 13.189871 test
4 Q0 G00-74-4030396 5 12.516583 test
4 Q0 G00-28-2286602 6 12.470680 test
4 Q0 G00-21-2229498 7 12.416529 test
4 Q0 G00-84-3349019 8 12.297861 test
4 Q0 G00-05-1218739 9 12.247684 test
4 Q0 G00-69-0005329 10 11.958622 test
4 Q0 G00-46-1439567 11 11.471069 test
4 Q0 G00-71-3454228 12 11.122462 test
4 Q0 G00-44-1482914 13 10.796131 test
4 Q0 G00-02-0735704 14 10.765544 test
4 Q0 G00-16-0059045 15 10.763972 test
4 Q0 G00-02-1720397 16 10.585347 test
4 Q0 G00-07-3064254 17 10.131027 test
4 Q0 G00-09-0774298 18 10.101838 test
4 Q0 G00-65-0162935 19 9.902224 test
4 Q0 G00-07-3695881 20 9.227164 test
4 Q0 G00-67-0152545 21 

### Q3 (c): Provide answer to Q3 (c) here [markdown cell]

I added LancasterStemmer,LowercaseFilter, and StopFilter to improve the performance. The baseline Whoosh system has a overall MAP score of 0.3455 of all query topics, which is higher than before(0.1971), and the map score of query topic 4 has been raised from 0.0312 to 0.5375. 

From above results, we can see that 3 over 4 relevant documnets has been retreived, and the false negative case from part (a) "G00-36-1275993" is retreived and ranked 1st, and the false positive file "G00-05-1218739" has a lower rank than before.

### Q3 (d): Provide answer to Q3 (d) here [markdown cell]

yes

### Q3 (e): Provide answer to Q3 (e) here [markdown cell]

yes

### Q3 (f): Provide answer to Q3 (f) here [markdown cell]

I think this change is good. The MAP score has been raised from 0.1971 to 0.3363. 7/15 query topics get higher MAP score than Q2, and the MAP score of 6/15 query topics didn't change, and only 2 query topics' MAP score get slightly lower than before.

## Question 4

In [0]:
# first, define a Schema for the index
Analyzer_Q4=RegexTokenizer() | LowercaseFilter() | StopFilter()| IntraWordFilter() |CustomFilter(LancasterStemmer().stem) | CustomFilter(WordNetLemmatizer().lemmatize)| CustomFilter(WordNetLemmatizer().lemmatize, 'v')

mySchema_Q4 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = Analyzer_Q4))

# now, create the index at the path INDEX_DIR based on the new schema
INDEX_Q4 = createIndex(mySchema_Q4)
addFilesToIndex(INDEX_Q4, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [0]:
QP_Q4 = QueryParser("file_content", schema=INDEX_Q4.schema, group=qparser.OrGroup)

In [0]:
#  Tuning b,k1
# for b in np.arange(0.5,1.0,0.05):
#   for k1 in np.arange(1.0,2.0,0.1):
#     SEARCHER_Q4_TEMP = INDEX_Q4.searcher(weighting=scoring.BM25F(B=round(b,2),K1=round(k1,2)))
#     print("b=",round(b,2),"k1=",round(k1,2))
#     pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q4, SEARCHER_Q4_TEMP) 

b= 0.5 k1= 1.0
map                      1       0.0578
map                      2       0.5303
map                      4       0.5579
map                      6       0.0244
map                      7       0.1310
map                      9       0.1250
map                      10      0.3333
map                      14      1.0000
map                      16      0.1452
map                      18      1.0000
map                      19      0.5000
map                      22      0.0500
map                      24      0.5000
map                      26      0.0971
map                      28      0.2250
map                      all     0.3518
b= 0.5 k1= 1.1
map                      1       0.0598
map                      2       0.5303
map                      4       0.5548
map                      6       0.0244
map                      7       0.1310
map                      9       0.2000
map                      10      0.3333
map                      14      1.0000
map       

**b=0.5,k1=1.6 with a highest overall MAP score of 0.4002**

In [0]:
SEARCHER_Q4 = INDEX_Q4.searcher(weighting=scoring.BM25F(B=0.5, K1=1.6))

In [0]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q4, SEARCHER_Q4) 

map                      1       0.0615
map                      2       0.5294
map                      4       0.5527
map                      6       0.0250
map                      7       0.2167
map                      9       0.2500
map                      10      0.3333
map                      14      1.0000
map                      16      0.1667
map                      18      1.0000
map                      19      0.5000
map                      22      0.0476
map                      24      1.0000
map                      26      0.0950
map                      28      0.2250
map                      all     0.4002


### Please answer the following questions here
####(a) A clear list of all final modifications made.  
I added IntraWordFilter and WordNetLemmatizer. I also changed the queryparser to use "or" instead "and" to search items and set the B value to be 0.5 and K1 value to be 1.6 implementing the BMF25 scoring algorithm. 


####(b)  Why each modification was made – how did it help?

IntraWordFilter is to help match different written representations of a concept, and WordNetLemmatizer is to convert a word to its base form. Adding these filters will make more documents contain relevant terms get retreived.

In Q3, the parser was set to use "and" by default, so if the document doesn't contain all query terms of a query topic, it won't be treated as a relevant document. To change the parser to use "or" instead, so that any of the terms may be present for a document to match, and more relevant documents will be retreived.

Adjusting the document scoring system may also improve the performance. After tuning the parameters of BMF25 scoring algorithm, I chose B=0.5 and K1=1.6 as final paramters.
####(c)  The  final  MAP  performance  that  these  modifications  attained.

The final MAP score is 0.4002

## Validation

In [0]:
# Run the following cells to make sure your code returns the correct value types

In [0]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Q2 Validation

In [0]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

Q2 Types Validated


### Q3 Validation

In [0]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

Q3 Types Validated


### Q4 Validation (Graduate Students)

In [0]:
GRAD_STUDENT=True

In [0]:
assert((not GRAD_STUDENT) or isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert((not GRAD_STUDENT) or isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert((not GRAD_STUDENT) or isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")

Q4 Types Validated
